In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

%matplotlib inline

os.chdir('/Users/chasedawson/dev/uva_equity_center/summer-sandbox/nfhl')
os.getcwd()

'/Users/chasedawson/dev/uva_equity_center/summer-sandbox/nfhl'

In [2]:
# read in eastern shore clipped coast data
cville_sp = {}
spatial_units = ['counties', 'tracts', 'blkgps', 'blocks']
path_to_data = '/../spatial_units/data/'
for su in spatial_units:
    cville_sp[su] = gpd.read_file(os.getcwd() + path_to_data + 'cville_{su}.shp'.format(su = su))

In [3]:
charlottesville_data_path = "./data/charlottesville/51003C_20210712"
albemarle_data_path = "./data/albemarle/51003C_20210712"
fluvanna_data_path = "./data/fluvanna/51003C_20210712"
greene_data_path = "./data/greene/51079C_20210323"
louisa_data_path = "./data/louisa/51109C_20200723"
nelson_data_path = "./data/nelson/51125C_20100618"

cville_data_paths = [
    albemarle_data_path,
    charlottesville_data_path, 
    fluvanna_data_path,
    greene_data_path,
    louisa_data_path,
    nelson_data_path
]

def read_paths(paths):
    return pd.concat([gpd.read_file(path) for path in paths])

paths = [os.path.join(data_path, "S_Fld_Haz_Ar.shp") for data_path in cville_data_paths]
cville_fhl = read_paths(paths)
cville_fhl.head()

,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,LEN_UNIT,VELOCITY,VEL_UNIT,AR_REVERT,AR_SUBTRV,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,geometry
0,51003C,1.1.1.0,51003C_788,NP,X,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,51003C_FIS1,"POLYGON ((-78.54919 38.12872, -78.54919 38.128..."
1,51003C,1.1.1.0,51003C_853,NP,A,None,T,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,51003C_FIS1,"POLYGON ((-78.70209 37.81493, -78.70225 37.815..."
2,51003C,1.1.1.0,51003C_865,NP,A,None,T,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,51003C_FIS1,"POLYGON ((-78.67918 38.02400, -78.67903 38.024..."
3,51003C,1.1.1.0,51003C_968,NP,AE,None,T,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,51003C_FIS1,"POLYGON ((-78.56976 38.06583, -78.56964 38.065..."
4,51003C,1.1.1.0,51003C_864,NP,A,None,T,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,51003C_FIS1,"POLYGON ((-78.52482 37.99485, -78.52485 37.994..."


In [4]:
def get_area(a):
    # get area of shp in km^2
    return round(a.geometry.to_crs("EPSG:3395").map(lambda p: p.area / 10**6).iloc[0], 6)

def get_intersection_area(a, b):
    """
    Computes the intersection of a and b and returns the intersection area and size of the intersected area
    as a percentage of the total area of a.
    
    Parameters
    ----------
    a : geopandas GeoDataFrame, required
    b : geopandas GeoDataFrame, required
    """
    intersection = gpd.overlay(a, b, how='intersection')
    if len(intersection) == 0:
        return {'area': 0, 'percent': 0}
    
    # get area of intersection in km^2
    intersect_area = get_area(intersection)
    
    # get area of original shp in km^2
    shp_area = get_area(a)
    
    # compute percentage of intersection of shp
    percentage = (intersect_area / shp_area) * 100
    
    return {'area': intersect_area, 'percent': percentage}

In [5]:
def get_perc_fld_zone_in_region(fhl, sp, logging = False):
    """
    Given National Flood Hazard Layer data (fhl) and spatial boundaries (sp) for counties, tracts, block groups, and blocks,
    this method computes the intersected area between the flood zone and specific geographic area to get an estimate of how much
    of that region is in what flood zone. 
    
    Parameters
    ----------
    fhl : GeoPandas GeoDataFrame, required
        National Flood Hazard Layer data.
        
    sp : dict, required
        Python dictionary containing spatial boundaries for counties, tracts, block groups, and blocks.
        Each spatial unit is a key for a GeoPandas GeoDataFrame with the spatial data.
        
    logging : bool, optional (default is False)
        Whether or not print statements should be executed for more information during runtime.
        
    Output
    ------
    GeoPandas GeoDataFrame
        Each row represents one flood zone type for a geographic region containing the information for how much of 
        that specific region was intersected by that flood zone type.
        
    """
    fld_zones = fhl.ZONE_SUBTY.value_counts().index.tolist()
    area_df = pd.DataFrame()
    for zone in fld_zones:
        if logging:
            print('Starting Zone: {zone}'.format(zone=zone))
        zone_df = fhl[fhl.ZONE_SUBTY == zone].dissolve()
        for su in ['counties', 'tracts', 'blkgps', 'blocks']:
            if logging:
                print('Starting Spatial Unit: {su}'.format(su = su))
                
            area_data = {
                'GEOID': [],
                'zone': [],
                'spatial_unit': [],
                'area': [],
                'perc': []
            }
            
            region = sp[su]
            
            if su == 'tracts':
                # get counties that were intersected by this flood zone
                intersected_areas = area_df[(area_df.zone == zone) & (area_df.spatial_unit == "counties") & (area_df.area > 0)]
                intersected_counties = sp['counties'][sp['counties'].GEOID.isin(intersected_areas.GEOID.values)]
                # filter region to only tracts within counties that have been intersected, reduces exploration
                region = region[region.COUNTYFP.isin(intersected_counties.COUNTYFP.values)]
            
            elif su == 'blkgps':
                # get tracts that were intersected by this flood zone
                intersected_areas = area_df[(area_df.zone == zone) & (area_df.spatial_unit == "tracts") & (area_df.area > 0)]
                intersected_tracts = sp['tracts'][sp['tracts'].GEOID.isin(intersected_areas.GEOID.values)]
                # filter region to only blkgps within tracts that have been intersected, reduces exploration
                region = region[region.TRACTCE.isin(intersected_tracts.TRACTCE.values)]
                
            elif su == 'blocks':
                # get tracts that were intersected by this flood zone
                intersected_areas = area_df[(area_df.zone == zone) & (area_df.spatial_unit == "tracts") & (area_df.area > 0)]
                intersected_tracts = sp['tracts'][sp['tracts'].GEOID.isin(intersected_areas.GEOID.values)]
                # filter region to only blocks that have been intersected, reduces exploration
                region = region[region.TRACTCE.isin(intersected_tracts.TRACTCE.values)]
                
            for i in tqdm(range(len(region))):
                shp = region.iloc[[i]]
                stats = get_intersection_area(shp, zone_df)
                
                # append data to area data
                area_data['GEOID'].append(shp.GEOID.iloc[0])
                area_data['zone'].append(zone)
                area_data['spatial_unit'].append(su)
                area_data['area'].append(stats['area'])
                area_data['perc'].append(stats['percent'])
                
            area_df = pd.concat([area_df, pd.DataFrame(area_data)])
            
    return area_df

In [ ]:
cville_area_df = get_perc_fld_zone_in_region(cville_fhl, cville_sp, logging = True)

Starting Zone: 0.2 PCT ANNUAL CHANCE FLOOD HAZARD


  0%|          | 0/6 [00:00<?, ?it/s]

Starting Spatial Unit: counties


  0%|          | 0/41 [00:00<?, ?it/s]

Starting Spatial Unit: tracts


...

  0%|          | 0/152 [00:00<?, ?it/s]

Starting Spatial Unit: blkgps


  0%|          | 0/10503 [00:00<?, ?it/s]

Starting Spatial Unit: blocks


 72%|███████▏  | 7551/10503 [3:41:38<1:26:52,  1.77s/it]